In [82]:
from utils import SOF_to_df

In [107]:
df = SOF_to_df("""Date        Status
2019-02-01  False
2019-02-02  True
2019-02-03  True
2019-02-04  False
2019-02-05  True
2019-02-06  True
2019-02-07  False
2019-02-08  False
2019-02-09  True
2019-02-10  True
2019-03-03  False
2019-03-04  True
2019-03-05  True
2019-03-06  True
2019-03-07  False
2019-03-08  True
2019-03-09  False""")


In [84]:
import pandas as pd

In [116]:
i=pd.MultiIndex(levels=[['foo','bar'],[1,2,3],df['Date']],codes=[[0]*10+[1]*7,[0,0,0,0,1,1,1,2,2,2,0,0,1,1,1,1,1],range(17)])

In [117]:
df = df.set_index(i).drop(['Date'],axis=1)

In [87]:
df['prev_Status'] = df['Status'].shift(1)

In [88]:
df['next_Status'] = df['Status'].shift(-1)

In [89]:
df

Status prev_Status next_Status
      Date                                     
foo 1 2019-02-01  False         NaN        True
      2019-02-02   True       False        True
      2019-02-03   True        True       False
      2019-02-04  False        True        True
    2 2019-02-05   True       False        True
      2019-02-06   True        True       False
      2019-02-07  False        True       False
    3 2019-02-08  False       False        True
      2019-02-09   True       False        True
      2019-02-10   True        True       False
bar 1 2019-03-03  False        True        True
      2019-03-04   True       False        True
    2 2019-03-05   True        True        True
      2019-03-06   True        True       False
      2019-03-07  False        True        True
      2019-03-08   True       False       False
      2019-03-09  False        True         NaN

In [93]:
df['Date'] = df.index.levels[2]

In [94]:
df.query('Status=="False"').query('next_Status=="True"')

Status prev_Status next_Status        Date
      Date                                                 
foo 1 2019-02-01  False         NaN        True  2019-02-01
      2019-02-04  False        True        True  2019-02-04
    3 2019-02-08  False       False        True  2019-02-08
bar 1 2019-03-03  False        True        True  2019-03-03
    2 2019-03-07  False        True        True  2019-03-07

In [95]:
df.query('prev_Status=="False"').query('Status=="True"').groupby(level=[0,1]).Date.agg(','.join)

bar  1    2019-03-04
     2    2019-03-08
foo  1    2019-02-02
     2    2019-02-05
     3    2019-02-09
Name: Date, dtype: object

In [96]:
df.query('Status=="True"').query('next_Status=="False"').groupby(level=[0,1]).Date.agg(','.join)

bar  2    2019-03-06,2019-03-08
foo  1               2019-02-03
     2               2019-02-06
     3               2019-02-10
Name: Date, dtype: object

In [110]:
df['Status']=df['Status'].map({'True':True,'False':False})

In [111]:
df.Status

1     False
2      True
3      True
4     False
5      True
6      True
7     False
8     False
9      True
10     True
11    False
12     True
13     True
14     True
15    False
16     True
17    False
Name: Status, dtype: bool

In [112]:
(~df.Status)

1      True
2     False
3     False
4      True
5     False
6     False
7      True
8      True
9     False
10    False
11     True
12    False
13    False
14    False
15     True
16    False
17     True
Name: Status, dtype: bool

In [122]:
s=(~df.Status).groupby(level=[0,1]).cumsum().reset_index()

In [130]:
s.groupby(['level_0','level_1','Status']).agg(['first','last']).groupby(['level_0','level_1']).agg(','.join)

Date  \
                                            first   
level_0 level_1                                     
bar     1                              2019-03-03   
        2        2019-03-05,2019-03-07,2019-03-09   
foo     1                   2019-02-01,2019-02-04   
        2                   2019-02-05,2019-02-07   
        3                              2019-02-08   

                                                   
                                             last  
level_0 level_1                                    
bar     1                              2019-03-04  
        2        2019-03-06,2019-03-08,2019-03-09  
foo     1                   2019-02-03,2019-02-04  
        2                   2019-02-06,2019-02-07  
        3                              2019-02-10